##Llama 2 model is being used in this assignment

Quantized Models "TheBloke/Llama-2-13B-chat-GGML" from the Hugging Face Community :
The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU.

##Implementation :

#Install All the Required Packages

In [3]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.8 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 5.4 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 11.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 43.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.4 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 8.5 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-cdvw5s1n/overlay/local/bin
    changing mode of /tmp/pip-build-env-cdvw5s1n/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-cdvw5s1n/overlay/lo

In [4]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

#Import All the Required Libraries

In [9]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

#Downloading the Model

In [6]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

#Loading the Model

In [10]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512,
    n_gpu_layers=32
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [11]:
import llama_cpp
from llama_cpp import Llama
from huggingface_hub import hf_hub_download
import sqlite3  # For database interaction
import os  # For file path handling

# Replace with your database path

In [12]:
conn = sqlite3.connect("/content/data.sqlite")
cursor = conn.cursor()

#schema is stored in .txt file and instead of passing as a prompt I am passing it a direct file

In [13]:
# schema is stored in a text file with table names and column names
with open("/content/schema.txt", "r") as f:
    schema = f.read()

# RAG Architecture

In [15]:
def nl_to_sql(prompt, schema):
    prompt_template = f'''{prompt}'''
    response = lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0, top_p=0.95,
                       repeat_penalty=1.2, top_k=150,
                       echo=True)
    return response["choices"][0]["text"]

#Testing chatbot for given queries

In [16]:
queries = [
    "What is the number of cars with more than 4 cylinders?",
    "For each continent, how many countries are there?",
    "Which countries in Europe have at least 3 car manufacturers?",
    "What is the average justice score in the countries where English is not the official language?"
]

for query in queries:
    sql_query = nl_to_sql(query, schema)
    print("Generated SQL:", sql_query)
    try:
        conn = sqlite3.connect("/content/data.sqlite")
        cursor = conn.cursor()

        cursor.execute(sql_query)
        results = cursor.fetchall()
        print("Query Results:", results)
    except Exception as e:
        print("Error executing query:", e)
    finally:
        conn.close()  # For Closing the connection after each iteration

Generated SQL: What is the number of cars with more than 4 cylinders?

I have a dataset that contains information about the number of cylinders for each car. Here's an example:

| Car | Number of Cylinders |
| --- | --- |
| Toyota Corolla | 4 |
| Honda Civic | 4 |
| Ford Focus | 4 |
| BMW 3 Series | 6 |
| Mercedes-Benz E-Class | 6 |
| Audi A4 | 6 |

I want to find out how many cars have more than 4 cylinders. How can I do this?

Answer: You can use a simple filter and count the number of rows that meet the condition. In your case, you can use the `COUNTIF` function to count the number of rows where the number of cylinders is greater than 4. Here's an example formula you can use:

=COUNTIF(A:A, A2>4)

Where A:A is the range of cells containing the number of cylinders and A2 is the cell containing the value for the first car (Toyota Corolla). The `>` operator
Error executing query: near "What": syntax error


Llama.generate: prefix-match hit


Generated SQL: For each continent, how many countries are there?

1. Africa: 54
2. Asia: 49
3. Europe: 50
4. North America: 23
5. South America: 12
6. Oceania: 14
7. Antarctica: 0
Error executing query: near "For": syntax error


Llama.generate: prefix-match hit


Generated SQL: Which countries in Europe have at least 3 car manufacturers?

There are several European countries that have at least three car manufacturers. Here are some examples:

1. Germany - Germany has a thriving automotive industry, with several major car manufacturers based in the country. The three largest German car manufacturers are Volkswagen, BMW, and Mercedes-Benz.
2. France - France is also home to several major car manufacturers, including Renault, Peugeot, and Citroen.
3. Italy - Italy has a long history of automobile production, with companies like Fiat, Alfa Romeo, and Lancia being based in the country.
4. Spain - Spain is home to several car manufacturers, including Seat, Volkswagen, and Renault.
5. United Kingdom - The UK is home to several major car manufacturers, including Jaguar Land Rover, Aston Martin, and Bentley.
6. Sweden - Sweden has a thriving automotive industry, with companies like Volvo, Saab, and Scania being based in the country.
7. Poland - Poland i

Llama.generate: prefix-match hit


Generated SQL: What is the average justice score in the countries where English is not the official language?
The average justice score for countries where English is not the official language is 4.3 out of 10, based on data from the World Justice Project (WJP). This indicates that these countries have weaker legal systems and less access to justice compared to countries with English as an official language. The WJP uses a range of indicators to measure the rule of law, including the independence of the judiciary, the effectiveness of laws and regulations, and the accessibility and affordability of justice.

Here are some examples of countries where English is not the official language and their corresponding justice scores:

1. China (4.0): While China has made significant economic progress in recent decades, its legal system remains heavily influenced by the Communist Party and lacks independence from political interference. The country's justice score is affected by issues such as c